In [137]:
import sys
sys.path.append("./classifier")

In [138]:
import os
import numpy as np
import torch
from sklearn import metrics
from networks import fcn1
from aggregators import LaplaceAggregator
from datasets import WebrequestDataset
from torch.utils.data import DataLoader, Subset, DataLoader
from pytorch_lightning.metrics.functional import stat_scores
from models import Student

In [139]:
!ls

classifier	      student_predictions.pt
data		      tasks
indices_250_a.pt      teacher_preds_250_a.pt
indices_250_small.pt  teacher_preds_250_small.pt
indices.pt	      teacher_thresholds_250_a.pt
notes		      teacher_thresholds_250_small.pt
Pipfile		      teacher_thresholds.pt
preds.pt	      wandb
README.md	      webrequest_data_analysis.ipynb
requirements.txt


In [140]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [141]:
num_workers = 4
aggregator = LaplaceAggregator(None)
dataset = WebrequestDataset(max_teacher_samples=None, max_student_train_queries=100000, num_workers=num_workers)
dataset.prepare_data()
dataset = dataset.train_data

In [187]:
eps = 10

def laplace_randomizer(row):
    scale = 2/eps
    noise = np.random.laplace(0, scale, row.shape)
    return row + noise

In [188]:
dataset.data

,/,javascript,.,core,_,2,-,m,default,php,...,es,title,c0,2565,2563,2569,2574,256f,2e,Etiqueta
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
233514,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
233515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
233516,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [191]:
dataset.data.loc[:, :500] = np.apply_along_axis(laplace_randomizer, 0, dataset.data.values[:, :500])
dataset.data

,/,javascript,.,core,_,2,-,m,default,php,...,es,title,c0,2565,2563,2569,2574,256f,2e,Etiqueta
0,0.402330,0.163454,-0.174181,-0.260681,-0.042050,-0.250720,0.147582,-0.117527,0.378677,0.158824,...,-0.204784,0.205166,-0.430915,-0.331801,-0.031817,-0.367155,-0.109164,-0.018504,-0.182697,0
1,0.121648,-0.027866,-0.014436,-0.183708,0.003092,-0.258950,0.016678,0.160752,0.093264,0.343358,...,-0.261413,0.346396,0.179508,-0.198188,-0.003301,0.072421,-0.056027,-0.223616,0.673254,0
2,0.323710,-0.082253,0.392754,-0.062300,0.263891,-0.039616,0.053854,0.040871,0.022189,-0.065998,...,0.028716,0.157262,-0.312881,-0.041787,-0.132009,-0.479509,0.032833,0.280399,0.010209,0
3,0.238826,-0.061918,-0.162094,0.257090,0.146824,-0.131914,0.422214,-0.329229,-0.123601,-0.305233,...,-0.320868,0.067109,0.287039,0.726392,0.016378,-0.041612,0.277770,-0.191940,-0.147569,0
4,-0.019984,0.172705,0.945801,0.076469,0.387535,0.687507,0.134160,-0.286811,-0.309020,0.080939,...,0.028112,0.017411,-0.026208,0.192903,0.258310,-0.277608,-0.161500,-0.373790,-0.260549,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233513,0.086960,-0.237628,-0.081476,0.031642,-0.050287,0.132985,-0.728673,0.022516,0.002642,0.321998,...,0.069256,-0.032521,0.381918,0.480285,-0.005639,0.362250,-0.012361,-0.060412,-0.047881,0
233514,-0.297493,-0.193556,0.046871,-0.244502,0.042476,-0.120988,0.060110,-0.184807,-0.007857,0.404407,...,-0.041711,0.592387,0.074345,0.036422,-0.466700,0.342867,0.012460,0.132762,-0.003149,0
233515,0.084421,-0.292105,-0.011455,0.583374,-0.129761,-0.043415,-0.624632,0.526226,0.084318,0.683214,...,0.013542,-0.086428,-0.078640,0.048740,-0.304504,-0.642063,0.007829,0.202554,0.395071,0
233516,-0.139345,-0.014848,0.847034,-0.522617,0.170083,-0.052347,-0.020478,0.117301,-0.354745,0.066925,...,-0.029589,0.097460,0.624603,0.022929,-0.194081,-0.276612,-0.229840,0.049376,0.053142,0


In [143]:
# Load teacher_networks

teacher_networks = []
teachers_path = "classifier/weights/teachers/2020-08-24-22:17-cute-tiger"
input_shape = [500]

for i, teacher_file in enumerate(sorted(os.listdir(teachers_path))):
    if teacher_file.startswith("teacher_"):
        teacher_network = fcn1(input_shape)
        teacher_network.load_state_dict(
            torch.load(f"{teachers_path}/{teacher_file}")
        )
        teacher_network.eval()
        teacher_network.to(device)
        teacher_networks.append(teacher_network)

In [144]:
indices = torch.load('./indices_250_a.pt')
# preds_b = torch.load('preds.pt', map_location=device)
# teacher_thresholds_b = torch.load('teacher_thresholds.pt', map_location=device)
print(indices.shape)

(233518,)


In [177]:
counter = 1
def add_one(row):
    return row * row

a = np.random.random((2, 3))
print(a)
print(np.apply_along_axis(add_one, 0, a))

[[0.45223681 0.5993732  0.51970238]
 [0.42448468 0.55262561 0.98522146]]
[[0.20451813 0.35924824 0.27009056]
 [0.18018724 0.30539506 0.97066133]]


In [161]:
# Calculate teacher_thresholds

teacher_thresholds = []
samples_per_teacher = 800
batch_size = 512

for i, teacher in enumerate(teacher_networks):
    subset = Subset(dataset, indices[i*samples_per_teacher:(i+1)*samples_per_teacher])
    dataloader = DataLoader(subset, batch_size=batch_size, num_workers=num_workers)

    y_pred = torch.zeros(0, dtype=torch.float).to(device)
    y_true = torch.zeros(0, dtype=torch.float).to(device)

    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        output = teacher(laplace_randomizer(images))

        y_pred = torch.cat((y_pred, output.reshape(output.shape[0])))
        y_true = torch.cat((y_true, labels.reshape(labels.shape[0])))

    fpr, tpr, thresholds = metrics.roc_curve(y_true.cpu(), y_pred.cpu().detach().numpy())
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]
    teacher_thresholds.append(optimal_threshold)

In [163]:
# measure threshold accuracy

for thresh in [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]:
    tp, fn, tn, fp, _ = stat_scores(y_true, (y_pred > thresh).long(), class_index=1)
    tp = tp.item()
    tn = tn.item()
    fp = fp.item()
    fn = fn.item()
    print(f"{thresh}:", tp/(tp+fn),tn/(tn+fp),tp, fn, tn, fp)

0.05: 0.3103448275862069 0.8443579766536965 9 20 651 120
0.1: 0.3103448275862069 0.8586251621271076 9 20 662 109
0.15: 0.27586206896551724 0.8664072632944229 8 21 668 103
0.2: 0.27586206896551724 0.8715953307392996 8 21 672 99
0.25: 0.2413793103448276 0.8767833981841764 7 22 676 95
0.3: 0.2413793103448276 0.8780804150453956 7 22 677 94
0.35: 0.2413793103448276 0.8819714656290532 7 22 680 91
0.4: 0.2413793103448276 0.8832684824902723 7 22 681 90
0.45: 0.2413793103448276 0.8897535667963683 7 22 686 85
0.5: 0.2413793103448276 0.893644617380026 7 22 689 82
0.55: 0.2413793103448276 0.8975356679636836 7 22 692 79
0.6: 0.2413793103448276 0.8988326848249028 7 22 693 78
0.65: 0.2413793103448276 0.8988326848249028 7 22 693 78
0.7: 0.2413793103448276 0.9001297016861219 7 22 694 77
0.75: 0.2413793103448276 0.9001297016861219 7 22 694 77
0.8: 0.2413793103448276 0.9040207522697795 7 22 697 74
0.85: 0.2413793103448276 0.9092088197146563 7 22 701 70
0.9: 0.2413793103448276 0.9118028534370947 7 22 703 

In [26]:
 # Get student train dataloader

nr_samples_for_teacher_thresholds = 250*samples_per_teacher
nr_train_samples = 15000

train_idxs = indices[nr_samples_for_teacher_thresholds:nr_samples_for_teacher_thresholds+nr_train_samples]
train_subset = Subset(dataset, train_idxs)
dataloader = DataLoader(train_subset, batch_size=batch_size, num_workers=num_workers)

In [27]:
type(teacher_networks[0])

networks.fcn1.Classifier

In [29]:
# Make teacher binary predictions

preds = torch.zeros((len(teacher_networks), len(train_subset)), dtype=torch.int)

for i, teacher in enumerate(teacher_networks[:2]):
    teacher.eval()
    teacher_preds = torch.zeros(0, dtype=torch.int).to(device)

    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        output = teacher(images)
        y_hat = (output >= teacher_thresholds[i]).type(torch.int)
        teacher_preds = torch.cat((teacher_preds, y_hat.reshape(y_hat.shape[0])))

    print(i)
    preds[i] = teacher_preds

0
1


In [49]:
images[2]

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.3512, 0.1992, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2856,
        0.0000, 0.0000, 0.0000, 0.0000, 0.2580, 0.2581, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.2574, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3774,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2590, 0.0000,
        0.0000, 0.0000, 0.4811, 0.0000, 

In [44]:
labels.shape

torch.Size([152, 1])

In [33]:
# Calculate preds accuracy

y_true = torch.Tensor(dataset.data.loc[train_idxs, "Etiqueta"].values)
# y_true = y_true.to(device)

precision_scores = np.array([0, 0, 0, 0, 0, 0])
for i in range(len(preds)):
    tp, fn, tn, fp, _ = stat_scores(y_true, preds[i], class_index=1)
    tp = tp.item()
    tn = tn.item()
    fp = fp.item()
    fn = fn.item()
    precision_scores = np.vstack((precision_scores, np.array([tp/(tp+fn), tn/(tn+fp), tp, fn, tn, fp])))
precision_scores = np.delete(precision_scores, 0, 0)

In [75]:
precision_scores

array([[8.18383167e-01, 9.67614783e-01, 1.47800000e+03, 3.28000000e+02,
        3.06850000e+04, 1.02700000e+03],
       [7.54706534e-01, 9.78966953e-01, 1.36300000e+03, 4.43000000e+02,
        3.10450000e+04, 6.67000000e+02],
       [8.75415282e-01, 9.28260595e-01, 1.58100000e+03, 2.25000000e+02,
        2.94370000e+04, 2.27500000e+03],
       ...,
       [9.08084164e-01, 9.20755550e-01, 1.64000000e+03, 1.66000000e+02,
        2.91990000e+04, 2.51300000e+03],
       [8.43853821e-01, 9.63546922e-01, 1.52400000e+03, 2.82000000e+02,
        3.05560000e+04, 1.15600000e+03],
       [8.92026578e-01, 9.35513370e-01, 1.61100000e+03, 1.95000000e+02,
        2.96670000e+04, 2.04500000e+03]])

In [79]:
new_labels.shape

(33518,)

In [34]:
# Aggregate teacher votes

new_labels = np.array([]).astype(int)
teacher_preds = preds.cpu().numpy()
gamma = 0.05
classes_count = 2

for image_preds in np.transpose(teacher_preds):
    label_counts = np.bincount(image_preds, minlength=classes_count).astype(float)
    label_counts += np.random.laplace(0, 1 / gamma, classes_count)

    new_label = np.argmax(label_counts)
    new_labels = np.append(new_labels, new_label)

In [82]:
teacher_preds.shape

(249, 33518)

In [15]:
torch.save(teacher_thresholds, 'teacher_thresholds_250_large.pt')
torch.save(indices, 'indices_250_large.pt')
torch.save(teacher_preds, 'teacher_preds_250_large.pt')

In [136]:
preds_default = (y_pred >= 0.5).type(torch.int)
preds_78 = (y_pred >= 0.0078).type(torch.int)
preds_35 = (y_pred >= 0.0035192405).type(torch.int)

In [170]:
y_pred.shape

torch.Size([1000])

In [139]:
preds_default[:30]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 1, 0, 0, 0, 0], dtype=torch.int32)

In [140]:
preds_35[:30]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 1, 0, 0, 0, 0], dtype=torch.int32)

In [141]:
preds_78[:30]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 1, 0, 0, 0, 0], dtype=torch.int32)

In [143]:
(y_true == preds_78).sum()

tensor(95128)

In [144]:
(y_true == preds_default).sum()

tensor(97048)

In [145]:
(y_true == preds_35).sum()

tensor(94273)

In [160]:
tp, fp, tn, fn, _ = stat_scores(preds_default, y_true, class_index=1)
tp, fp, tn, fn = tp.item(), fp.item(), tn.item(), fn.item()
tpr = tp / (tp + fn)
tp, fp, tn, fn, tpr

(3996, 1676, 93052, 1276, 0.7579666160849773)

In [161]:
tp, fp, tn, fn, _ = stat_scores(preds_35, y_true, class_index=1)
tp, fp, tn, fn = tp.item(), fp.item(), tn.item(), fn.item()
tpr = tp / (tp + fn)
tp, fp, tn, fn, tpr

(4649, 5104, 89624, 623, 0.8818285280728376)

In [162]:
tp, fp, tn, fn, _ = stat_scores(preds_78, y_true, class_index=1)
tp, fp, tn, fn = tp.item(), fp.item(), tn.item(), fn.item()
tpr = tp / (tp + fn)
tp, fp, tn, fn, tpr

(4537, 4137, 90591, 735, 0.8605842185128983)

In [35]:
import pandas as pd

In [36]:
data = pd.read_csv("data/WebRequests/unigrams/train.csv", sep=" ", index_col=0)
data

,/,javascript,.,core,_,2,-,m,default,php,...,es,title,c0,2565,2563,2569,2574,256f,2e,Etiqueta
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
233514,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
233515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
233516,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [38]:
data[:10].corr()['Etiqueta']

/            NaN
javascript   NaN
.            NaN
core         NaN
_            NaN
              ..
2569         NaN
2574         NaN
256f         NaN
2e           NaN
Etiqueta     NaN
Name: Etiqueta, Length: 501, dtype: float64

In [59]:
import numpy as np
from sklearn import metrics

y = np.array([0, 1, 0, 1])
scores = np.array([0.1, 0.9, 0.15, 0.8])
fpr, tpr, thresholds = metrics.roc_curve(y, scores)

In [60]:
fpr

array([0., 0., 0., 1.])

In [61]:
tpr

array([0. , 0.5, 1. , 1. ])

In [62]:
thresholds

array([1.9, 0.9, 0.8, 0.1])